# **Scikit-Learn 기반 회귀분석**

In [ ]:
# 사용할 라이브러리 불러오기
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## **Scikit-Learn 라이브러리 불러오기**


In [ ]:
import sklearn

### 라이브러리 버전 확인

In [ ]:
sklearn.__version__

## **회귀분석 Dataset 불러오기**


In [ ]:
from sklearn import datasets

In [ ]:
data = datasets.load_diabetes(as_frame=True)

### 독립변수 및 종속변수 할당

In [ ]:
x, y = data.data, data.target

#### 독립변수 Column 확인

In [ ]:
inputFeatures = x.columns
print(inputFeatures)

## **데이터 전처리 진행**

### Min-Max Normalization 진행

In [ ]:
from sklearn.preprocessing import MinMaxScaler

### **Scikit-Learn 모듈 특성**



1.   Class Instance 생성
2.   fit() Method 실행
3.   transform() Method 실행



In [ ]:
x, y = np.array(x), np.array(y) # Numpy Array 자료 구조로 변환

#### 독립변수 전처리 진행

In [ ]:
mmScalerX = MinMaxScaler()
mmScalerX.fit(x)
x = mmScalerX.transform(x) # MIn-Max Normalization 적용

#### 종속변수 전처리 진행

In [ ]:
mmScalerY = MinMaxScaler()
mmScalerY.fit(y.reshape(-1, 1)) # 1차원에서 2차원으로 차원 증가
y = mmScalerY.transform(y.reshape(-1, 1)).reshape(-1) # 2차원에서 1차원으로 차원 축소

## **Dataset 샘플링 진행**

### Scikit-Learn의 **train_test_split** 활용

In [ ]:
from sklearn.model_selection import train_test_split

#### Dataset을 나누는 비율과 Shuffling후 일정한 결과를 얻기 위한 Seed 입력

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
print(xTrain.shape, yTrain.shape)
print(xTest.shape, yTest.shape)

## **다중 선형 회귀분석 비교**

In [ ]:
def getParameter(x: np.array, y: np.array) :
  xT = np.transpose(x)
  output = np.matmul(np.matmul(np.linalg.inv(np.matmul(xT, x)), xT), y)

  return output

In [ ]:
betaHat = getParameter(xTrain, yTrain)

In [ ]:
yTestHat = np.matmul(xTest, betaHat)

### **다중 선형 회귀 모델의 정성적 및 정량적 평가 진행**

In [ ]:
def plotBarChart(yTest, yTestHat) :
  fig, ax = plt.subplots(figsize=(20,10))
  idx = np.asarray([i for i in range(50)])
  width = 0.2

  ax.bar(idx, yTest[:50], width=width)
  ax.bar(idx+width, yTestHat[:50], width=width)
  ax.set_xticks(idx)
  ax.legend(["Ground Truth", "Prediction"])
  ax.set_xlabel("# samples")
  ax.set_ylabel("Value")

  fig.tight_layout()
  plt.show()

In [ ]:
def RMSE(yHat:np.array, y:np.array) :
  output = np.sqrt(np.mean(np.power(y - yHat, 2)))

  return output

#### 원래 차원으로 복원하기 위핸 Affine Transformation 진행

In [ ]:
yTest = mmScalerY.inverse_transform(yTest.reshape(-1, 1)).reshape(-1)
yTestHat = mmScalerY.inverse_transform(yTestHat.reshape(-1, 1)).reshape(-1)

In [ ]:
plotBarChart(yTest, yTestHat)

In [ ]:
RMSE(yTestHat, yTest)

## **의사 결정 트리 기반 회귀분석**

### Scikit-Learn의 **DecisionTreeRegressor** 활용

In [ ]:
from sklearn.tree import DecisionTreeRegressor



1.   Class Instance 생성
2.   fit() Method 실행
3.   predict() Method 실행



In [ ]:
dtReg = DecisionTreeRegressor(random_state=42)
dtReg.fit(xTrain, yTrain)
yTestHat = dtReg.predict(xTest)

### **입력 피처 (독립변수) 중요도 확인**

#### Seaborn 라이브러리 활용

In [ ]:
import seaborn as sns

In [ ]:
def visualizeImportances(data:pd.Series) :
  plt.figure(figsize=(8,6))
  plt.title("Feature Importances")
  plt.xlabel("Importance")
  plt.ylabel("Feature")
  sns.barplot(x=data, y=data.index)
  plt.show()

In [ ]:
visualizeImportances(pd.Series(dtReg.feature_importances_, index=inputFeatures).sort_values(ascending=False))

#### **Decision Tree의 Component 시각화**

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plt.figure(figsize=(24,18))
plot_tree(dtReg)
plt.show()

### **의사 결정 트리 기반 회귀분석 모델의 정성적 및 정량적 평가 진행**

#### 원래 차원으로 복원하기 위핸 Affine Transformation 진행

In [ ]:
yTestHat = mmScalerY.inverse_transform(yTestHat.reshape(-1, 1)).reshape(-1)

In [ ]:
plotBarChart(yTest, yTestHat)

In [ ]:
RMSE(yTestHat, yTest)

## **Random Forest (Ensemble 모델) 기반 회귀분석**

### Scikit-Learn의 **RandomForestRegressor** 활용

In [ ]:
from sklearn.ensemble import RandomForestRegressor



1.   Class Instance 생성
2.   fit() Method 실행
3.   predict() Method 실행



In [ ]:
rfReg = RandomForestRegressor(random_state=42)
rfReg.fit(xTrain, yTrain)
yTestHat = rfReg.predict(xTest)

### **입력 피처 (독립변수) 중요도 확인**

In [ ]:
visualizeImportances(pd.Series(rfReg.feature_importances_, index=inputFeatures).sort_values(ascending=False))

### **Random Forest 기반 회귀분석 모델의 정성적 및 정량적 평가 진행**

#### 원래 차원으로 복원하기 위핸 Affine Transformation 진행

In [ ]:
yTestHat = mmScalerY.inverse_transform(yTestHat.reshape(-1, 1)).reshape(-1)

In [ ]:
plotBarChart(yTest, yTestHat)

In [ ]:
RMSE(yTestHat, yTest)